In [3]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [256]:
pip install oauth2client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

In [5]:
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
]

In [6]:
GOOGLE_KEY_FILE = 'gsheets_key.json'

In [7]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials)

In [8]:
wokbook_key = '1mGcKJNUJJvVbN4ycStlQM6PIyWe7ZX1TSSu0OEjH6eY'
workbook = gc.open_by_key(wokbook_key)

In [9]:
sheet = workbook.get_worksheet(2)
values = sheet.get_all_values()

In [10]:
covid_data = pd.DataFrame(values[1:],columns=values[0])

In [304]:
covid_data.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416
0,Adams County,18001,,,,,1,1,1,1,...,2,2,2,4,4,5,5,6,6,6
1,Allen County,18003,,,,,,,,,...,87,95,105,119,136,147,156,167,180,224
2,Bartholomew County,18005,,,,,,,,,...,55,66,83,83,108,110,110,110,113,116
3,Benton County,18007,,,,,,,,,...,0,0,0,0,0,2,2,2,3,4
4,Blackford County,18009,,,,,,,,,...,4,4,4,4,5,5,5,6,6,6


In [11]:
Allen_Hamilton_Hendricks_covid = covid_data.loc[covid_data['County'].isin(['Allen County', 'Hamilton County', 'Hendricks County'])]

In [12]:
Allen_Hamilton_Hendricks_covid.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416
1,Allen County,18003,,,,,,,,,...,87,95,105,119,136,147,156,167,180,224
28,Hamilton County,18057,,,,,,,,,...,363,392,395,431,443,478,490,499,510,520
31,Hendricks County,18063,,,1,2,2,2,2,2,...,225,243,256,287,301,315,330,350,360,378


In [13]:
Allen_Hamilton_Hendricks_covid.columns

Index(['County', 'GEOGID', 'd_20200306', 'd_20200307', 'd_20200308',
       'd_20200309', 'd_20200310', 'd_20200311', 'd_20200312', 'd_20200313',
       'd_20200314', 'd_20200315', 'd_20200316', 'd_20200317', 'd_20200318',
       'd_20200319', 'd_20200320', 'd_20200321', 'd_20200322', 'D_20200323',
       'D_20200324', 'D_20200325', 'D_20200326', 'D_20200327', 'D_20200328',
       'D_20200329', 'D_20200330', 'D_20200331', 'D_20200401', 'D_20200402',
       'D_20200403', 'D_20200404', 'D_20200405', 'D_20200406', 'D_20200407',
       'D_20200408', 'D_20200409', 'D_20200410', 'D_20200411', 'D_20200412',
       'D_20200413', 'D_20200414', 'D_20200415', 'D_20200416'],
      dtype='object')

In [14]:
djv_melt = Allen_Hamilton_Hendricks_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")

In [15]:
djv_melt['Date'] = djv_melt['Date'].map(lambda x: x.lstrip('dD_'))

In [16]:
djv_melt.head()

,County,GEOGID,Date,Count
0,Allen County,18003,20200306,
1,Hamilton County,18057,20200306,
2,Hendricks County,18063,20200306,
3,Allen County,18003,20200307,
4,Hamilton County,18057,20200307,


In [18]:
import numpy as np
Allen_covid = djv_melt.loc[djv_melt['County'].isin(['Allen County'])]
Allen_covid = Allen_covid.reset_index(drop=True) 
Allen_covid.replace('', np.nan, inplace=True)
Allen_covid.fillna(0, inplace=True)
Allen_covid['Count'] = Allen_covid['Count'].astype(int)

In [318]:
Allen_covid.describe

<bound method NDFrame.describe of           County GEOGID      Date  Count
0   Allen County  18003  20200306      0
1   Allen County  18003  20200307      0
2   Allen County  18003  20200308      0
3   Allen County  18003  20200309      0
4   Allen County  18003  20200310      0
5   Allen County  18003  20200311      0
6   Allen County  18003  20200312      0
7   Allen County  18003  20200313      0
8   Allen County  18003  20200314      0
9   Allen County  18003  20200315      0
10  Allen County  18003  20200316      0
11  Allen County  18003  20200317      0
12  Allen County  18003  20200318      0
13  Allen County  18003  20200319      0
14  Allen County  18003  20200320      1
15  Allen County  18003  20200321      4
16  Allen County  18003  20200322      4
17  Allen County  18003  20200323      6
18  Allen County  18003  20200324      7
19  Allen County  18003  20200325      7
20  Allen County  18003  20200326      8
21  Allen County  18003  20200327     21
22  Allen County  18003

In [19]:
import pickle
Allen_covid.to_pickle("Allen_covid.pkl")

In [20]:
Allen_covid.to_json('static/data/Allen_covid.json')

In [21]:
import numpy as np
Hamilton_covid = djv_melt.loc[djv_melt['County'].isin(['Hamilton County'])]
Hamilton_covid = Hamilton_covid.reset_index(drop=True) 
Hamilton_covid.replace('', np.nan, inplace=True)
Hamilton_covid.fillna(0, inplace=True)
Hamilton_covid['Count'] = Hamilton_covid['Count'].astype(int)

In [23]:
Hamilton_covid.describe

<bound method NDFrame.describe of              County GEOGID      Date  Count
0   Hamilton County  18057  20200306      0
1   Hamilton County  18057  20200307      0
2   Hamilton County  18057  20200308      0
3   Hamilton County  18057  20200309      0
4   Hamilton County  18057  20200310      0
5   Hamilton County  18057  20200311      0
6   Hamilton County  18057  20200312      0
7   Hamilton County  18057  20200313      0
8   Hamilton County  18057  20200314      0
9   Hamilton County  18057  20200315      1
10  Hamilton County  18057  20200316      1
11  Hamilton County  18057  20200317      1
12  Hamilton County  18057  20200318      2
13  Hamilton County  18057  20200319      2
14  Hamilton County  18057  20200320      5
15  Hamilton County  18057  20200321     10
16  Hamilton County  18057  20200322     18
17  Hamilton County  18057  20200323     21
18  Hamilton County  18057  20200324     25
19  Hamilton County  18057  20200325     30
20  Hamilton County  18057  20200326     4

In [24]:
import pickle
Hamilton_covid.to_pickle("Hamilton_covid.pkl")

In [25]:
Hamilton_covid.to_json('static/data/Hamilton_covid.json')

In [29]:
import numpy as np
Hendricks_covid = djv_melt.loc[djv_melt['County'].isin(['Hendricks County'])]
Hendricks_covid = Hendricks_covid.reset_index(drop=True) 
Hendricks_covid.replace('', np.nan, inplace=True)
Hendricks_covid.fillna(0, inplace=True)
Hendricks_covid['Count'] = Hendricks_covid['Count'].astype(int)

In [31]:
Hendricks_covid.describe

<bound method NDFrame.describe of               County GEOGID      Date  Count
0   Hendricks County  18063  20200306      0
1   Hendricks County  18063  20200307      0
2   Hendricks County  18063  20200308      1
3   Hendricks County  18063  20200309      2
4   Hendricks County  18063  20200310      2
5   Hendricks County  18063  20200311      2
6   Hendricks County  18063  20200312      2
7   Hendricks County  18063  20200313      2
8   Hendricks County  18063  20200314      2
9   Hendricks County  18063  20200315      2
10  Hendricks County  18063  20200316      3
11  Hendricks County  18063  20200317      3
12  Hendricks County  18063  20200318      4
13  Hendricks County  18063  20200319      4
14  Hendricks County  18063  20200320      4
15  Hendricks County  18063  20200321      6
16  Hendricks County  18063  20200322      8
17  Hendricks County  18063  20200323     12
18  Hendricks County  18063  20200324     13
19  Hendricks County  18063  20200325     15
20  Hendricks County 

In [32]:
import pickle


In [34]:
Hendricks_covid.to_pickle("Hendricks_covid.pkl")

In [35]:
Hendricks_covid.to_json('static/data/Hendricks_covid.json')